![DoctorBert](https://snag.gy/oF84Gk.jpg)


![DoctorBert](https://i.ytimg.com/vi/nPemP-Q0Xn8/hqdefault.jpg)

This is a Colab Demo of our DocProduct Tensorflow 2.0 Hackathon Project

Project details can be seen on our Github repo
https://github.com/Santosh-Gupta/DocProduct

and our Devpost page
https://devpost.com/software/nlp-doctor

In order to fit the computational limits of Google Colab, we had to reduce the amount of searchable Q&A datapoints from ~700,000 to ~200,000.
We also converted our search embeddings from float32 to float16, although this does not seem to dimish the quality of search (*but that's a whole other product to explore*)

Instructions:

-Run the first cell to install TF2.0 and FAISS

-Run the second cell to download the models and data

-Run the third cell to load the weights and data

-Input your question and search parameters into the fourth cell to play around with our model results




In [5]:
#@title Install Faiss, TF 2.0, and our Github. Double Click to see code

#To use CPU FAISS use
!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
#To use GPU FAISS use
#!wget  https://anaconda.org/pytorch/faiss-gpu/1.2.1/download/linux-64/faiss-gpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

!pip install keras
!pip install tensorflow-gpu==2.0.0-alpha0
!pip install tensorflow-cpu
import tensorflow as tf
!pip install https://github.com/re-search/DocProduct/archive/v0.2.0_dev.zip
!pip install gpt2-estimator
!pip install pyarrow


--2019-11-25 14:03:39--  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.92.24, 104.17.93.24
Connecting to anaconda.org (anaconda.org)|104.17.92.24|:443... connected.
ERROR: cannot verify anaconda.org's certificate, issued by 'CN=CloudFlare Inc ECC CA-2,O=CloudFlare\\, Inc.,L=San Francisco,ST=CA,C=US':
  Unable to locally verify the issuer's authority.
To connect to anaconda.org insecurely, use `--no-check-certificate'.
tar: Error opening archive: Failed to open 'faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2'
cp: cannot stat 'lib/python3.6/site-packages/*': No such file or directory


  Created wheel for opt-einsum: filename=opt_einsum-3.1.0-cp36-none-any.whl size=63926 sha256=749b4581ffe39ba89a86601c1ce479cbeb27747e16d5656cac0f4a12528bea30
  Stored in directory: C:\Users\imroy\AppData\Local\pip\Cache\wheels\2c\b1\94\43d03e130b929aae7ba3f8d15cbd7bc0d1cb5bb38a5c721833
  Created wheel for wrapt: filename=wrapt-1.11.2-cp36-none-any.whl size=21432 sha256=3c5d8e618cc40c847aa1c886372b84a5e8ca9aab8dbfcbb28269c8874533ea15
  Stored in directory: C:\Users\imroy\AppData\Local\pip\Cache\wheels\d7\de\2e\efa132238792efb6459a96e85916ef8597fcb3d2ae51590dfd
Successfully built opt-einsum wrapt
  Found existing installation: numpy 1.14.5
    Uninstalling numpy-1.14.5:
      Successfully uninstalled numpy-1.14.5


ERROR: tensorflow 1.13.1 has requirement tensorboard<1.14.0,>=1.13.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\imroy\\anaconda3\\lib\\site-packages\\~~mpy\\.libs\\libopenblas.BNVRK7633HSX7YVO2TADGR4A5KEKXJAW.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



ImportError: Traceback (most recent call last):
  File "C:\Users\imroy\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\imroy\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\imroy\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\imroy\Anaconda3\lib\imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\imroy\Anaconda3\lib\imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: The specified module could not be found.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for

In [0]:
#@title Downaload all model checkpoints, and question/answer data. Double click to see this code

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                
import os
import requests

import urllib.request

# Download the file from `url` and save it locally under `file_name`:
urllib.request.urlretrieve('https://github.com/naver/biobert-pretrained/releases/download/v1.0-pubmed-pmc/biobert_v1.0_pubmed_pmc.tar.gz', 'BioBert.tar.gz')

if not os.path.exists('BioBertFolder'):
    os.makedirs('BioBertFolder')
    
import tarfile
tar = tarfile.open("BioBert.tar.gz")
tar.extractall(path='BioBertFolder/')
tar.close()

file_id = '1uCXv6mQkFfpw5txGnVCsl93Db7t5Z2mp'

download_file_from_google_drive(file_id, 'Float16EmbeddingsExpanded5-27-19.pkl')

file_id = 'https://onedrive.live.com/download?cid=9DEDF3C1E2D7E77F&resid=9DEDF3C1E2D7E77F%2132792&authkey=AEQ8GtkcDbe3K98'
    
urllib.request.urlretrieve( file_id, 'DataAndCheckpoint.zip')

if not os.path.exists('newFolder'):
    os.makedirs('newFolder')

import zipfile
zip_ref = zipfile.ZipFile('DataAndCheckpoint.zip', 'r')
zip_ref.extractall('newFolder')
zip_ref.close()

In [1]:
#@title Load model weights and Q&A data. Double click to see code

from docproduct.predictor import RetreiveQADoc

pretrained_path = 'BioBertFolder/biobert_v1.0_pubmed_pmc/'
# ffn_weight_file = None
bert_ffn_weight_file = 'newFolder/models/bertffn_crossentropy/bertffn'
embedding_file = 'Float16EmbeddingsExpanded5-27-19.pkl'

doc = RetreiveQADoc(pretrained_path=pretrained_path,
ffn_weight_file=None,
bert_ffn_weight_file=bert_ffn_weight_file,
embedding_file=embedding_file)

Failed to load GPU Faiss: No module named 'faiss.swigfaiss_gpu'
Faiss falling back to CPU-only.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3

In [7]:
#@title Type in your question (512 word limit) and search search parameters

question_text = "I am having stomach pain I had some street food yesterday"  #@param {type:"string"}

search_similarity_by = 'answer'  #@param ['answer', "question"]

number_results_toReturn=5 #@param {type:"number"}

answer_only=True #@param ["False", "True"] {type:"raw"}

returned_results = doc.predict( question_text ,
                  search_by=search_similarity_by, topk=number_results_toReturn, answer_only=answer_only)

print('')
for jk in range(len(returned_results)):
    print("Result ", jk+1)
    print(returned_results[jk])
    print('')


1it [00:01,  1.41s/it]


Result  1
If you're not having stabbing pains in your throat, you're fine.  The chance of it causing harm as it travels around is extremely small, because your stomach acid will make it swell and make it soft and squishy (if it's wooden)

No need to do anything,lol no problem.  To be honest with you , you probably won't know it passed.  It'll be so squishy and soft that it will probably be inside of a feces.  The number of days varies a lot, but is somewhere between 1-4 days depending on many factors like age and diet, etc

Try not to worry. you will be fine 

Result  2
I think it would be very unlikely for you to develop a peptic ulcer in that short amount of time. A more likely explanation is that your recent illness is causing stomach upset/inflammation (gastritis). Especially if you were vomiting a lot, things could be pretty irritated and you might still have some lingering symptoms from the illness. Ibuprofen can also be rough on the stomach.

I'd say wait it out for now, try to

In [4]:
print("hello")

hello
